<a href="https://colab.research.google.com/github/sutharimanikanta/Golab/blob/main/Age_gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openCv

In [ ]:
# Import required modules
import cv2 as cv  # OpenCV library
import math  # Mathematical operations
import time  # Time-related operations
import argparse  # Argument parsing

# Function to detect faces and return bounding boxes
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()  # Create a copy of the frame
    frameHeight = frameOpencvDnn.shape[0]  # Height of the frame
    frameWidth = frameOpencvDnn.shape[1]  # Width of the frame
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)  # Preprocess image

    # Set input for the neural network and perform forward pass
    net.setInput(blob)
    detections = net.forward()
    bboxes = []  # List to store bounding boxes
    # Iterate through detected faces
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]  # Confidence of detection
        if confidence > conf_threshold:  # Check if confidence is above threshold
            # Calculate bounding box coordinates
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])  # Add bounding box coordinates to list
            # Draw bounding box on the frame
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes  # Return annotated frame and bounding boxes

# Argument parsing
parser = argparse.ArgumentParser(description='Use this script to run age and gender recognition using OpenCV.')
parser.add_argument('--input', help='Path to input image or video file. Skip this argument to capture frames from a camera.')
parser.add_argument("--device", default="cpu", help="Device to inference on")
args, unknown = parser.parse_known_args()
if unknown:
    print("Warning: Unknown arguments:", unknown)

args = parser.parse_args()  # Parse command-line arguments

# Load pre-trained models for face detection, age estimation, and gender recognition
faceProto = "/content/opencv_face_detector.pbtxt"  # Face detection model architecture
faceModel = "/content/opencv_face_detector_uint8.pb"  # Face detection model weights

ageProto = "/content/age_deploy.prototxt"  # Age estimation model architecture
ageModel = "/content/age_net.caffemodel"  # Age estimation model weights

genderProto = "/content/gender_deploy.prototxt"  # Gender recognition model architecture
genderModel = "/content/gender_net.caffemodel"  # Gender recognition model weights

# Mean values for preprocessing input images for age and gender prediction
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']  # Age categories
genderList = ['Male', 'Female']  # Gender categories

# Load neural networks for age and gender prediction
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

# Set backend and target for inference based on device
if args.device == "cpu":
    ageNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)
    genderNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)
    faceNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)
    print("Using CPU device")
elif args.device == "gpu":
    ageNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    ageNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
    genderNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    genderNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
    faceNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    faceNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")

# Open video file, image file, or camera stream
cap = cv.VideoCapture(args.input if args.input else 0)
padding = 20  # Padding for face bounding boxes
while cv.waitKey(1) < 0:  # Continue until key press
    # Read frame
    t = time.time()  # Record start time
    hasFrame, frame = cap.read()  # Read frame from video capture
    if not hasFrame:  # If no frame is captured, exit loop
        cv.waitKey()
        break

    # Detect faces and get bounding boxes
    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:  # If no faces are detected, skip processing
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:  # Iterate through detected faces
        face = frame[max(0, bbox[1]-padding):min(bbox[3]+padding, frame.shape[0]-1),
                     max(0, bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]  # Extract face region
        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)  # Preprocess face image
        genderNet.setInput(blob)  # Set input for gender prediction
        genderPreds = genderNet.forward()  # Perform gender prediction
        gender = genderList[genderPreds[0].argmax()]  # Get predicted gender
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))  # Print gender and confidence

        ageNet.setInput(blob)  # Set input for age prediction
        agePreds = ageNet.forward()  # Perform age prediction
        age = ageList[agePreds[0].argmax()]  # Get predicted age category
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))  # Print age and confidence

        label = "{},{}".format(gender, age)  # Combine gender and age labels
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)  # Draw label on frame
        cv.imshow("Age Gender Demo", frameFace)  # Display annotated frame
    print("time : {:.3f}".format(time.time() - t))  # Print processing time

# Commented section below is a CMake command for building OpenCV with CUDA support
# It's provided as a reference for compiling OpenCV with CUDA on your system
# cmake -DCMAKE_BUILD_TYPE=RELEASE -DCMAKE_INSTALL_PREFIX=~/opencv_gpu -DINSTALL_PYTHON_EXAMPLES=OFF -DINSTALL_C_EXAMPLES=OFF -DOPENCV_ENABLE_NONFREE=ON -DOPENCV_EXTRA_MODULES_PATH=~/cv2_gpu/opencv_contrib/modules -DPYTHON_EXECUTABLE=~/env/bin/python3 -DBUILD_EXAMPLES=ON -DWITH_CUDA=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON -DENABLE_FAST_MATH=ON -DCUDA_FAST_MATH=ON -DWITH_CUBLAS=ON -DCUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda-10.2 -DOpenCL_LIBRARY=/usr/local/cuda-10.2/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda-10.2/include/ ..


usage: colab_kernel_launcher.py [-h] [--input INPUT] [--device DEVICE]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-449ae5a9-4920-41ec-b83f-90ebb7e91aa8.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import cv2 as cv
import argparse
import time

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])

    return frameOpencvDnn, bboxes

def main(args):

    # Load pre-trained models for face detection, age estimation, and gender recognition
    faceProto = "/content/opencv_face_detector.pbtxt"  # Face detection model architecture
    faceModel = "/content/opencv_face_detector_uint8.pb"  # Face detection model weights

    ageProto = "/content/age_deploy.prototxt"  # Age estimation model architecture
    ageModel = "/content/age_net.caffemodel"  # Age estimation model weights

    genderProto = "/content/gender_deploy.prototxt"  # Gender recognition model architecture
    genderModel = "/content/gender_net.caffemodel"  # Gender recognition model weights

    MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
    ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
    genderList = ['Male', 'Female']

    faceNet = cv.dnn.readNet(faceModel, faceProto)
    ageNet = cv.dnn.readNet(ageModel, ageProto)
    genderNet = cv.dnn.readNet(genderModel, genderProto)

    if args.device == "gpu":
        faceNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
        faceNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
        ageNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
        ageNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
        genderNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
        genderNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

    cap = cv.VideoCapture(args.input if args.input else 0)
    padding = 20

    while cv.waitKey(1) < 0:
        t = time.time()
        hasFrame, frame = cap.read()
        if not hasFrame:
            cv.waitKey()
            break

        frameFace, bboxes = getFaceBox(faceNet, frame)
        if not bboxes:
            print("No face Detected, Checking next frame")
            continue

        for bbox in bboxes:
            face = frame[max(0, bbox[1]-padding):min(bbox[3]+padding, frame.shape[0]-1),
                         max(0, bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
            blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            genderNet.setInput(blob)
            genderPreds = genderNet.forward()
            gender = genderList[genderPreds[0].argmax()]
            print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

            ageNet.setInput(blob)
            agePreds = ageNet.forward()
            age = ageList[agePreds[0].argmax()]
            print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

            label = "{},{}".format(gender, age)
            cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
            cv.imshow("Age Gender Demo", frameFace)

        print("time : {:.3f}".format(time.time() - t))

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Use this script to run age and gender recognition using OpenCV.')
    parser.add_argument('--input', help='Path to input image or video file. Skip this argument to capture frames from a camera.')
    parser.add_argument("--device", default="cpu", choices=["cpu", "gpu"], help="Device to inference on")
    args = parser.parse_args()
    main(args)


usage: colab_kernel_launcher.py [-h] [--input INPUT] [--device {cpu,gpu}]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-449ae5a9-4920-41ec-b83f-90ebb7e91aa8.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import cv2 as cv
import time
import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes


parser = argparse.ArgumentParser(description='Use this script for face recognition using OpenCV.')
parser.add_argument('--input', help='Path to input image or video file. Skip this argument to capture frames from a camera.')
parser.add_argument("--device", default="cpu", help="Device to inference on")

# Parse only recognized arguments
args, _ = parser.parse_known_args()

faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

# Load face detection model
faceNet = cv.dnn.readNet(faceModel, faceProto)

if args.device == "cpu":
    faceNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)
    print("Using CPU device")
elif args.device == "gpu":
    faceNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    faceNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")

# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(args.input if args.input else 0)
padding = 20

while cv.waitKey(1) < 0:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    # Detect faces in the frame
    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # Draw bounding box around the face
        cv.imshow("Face Detection", frameFace)

    print("Time taken: {:.3f} seconds".format(time.time() - t))

# Release the video capture object
cap.release()
cv.destroyAllWindows()


Using CPU device
